In [1]:
%load_ext jupyternotify

# import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import main
# import timeit
import joblib
import importlib
import xgboost as xgb

from tqdm.auto import tqdm
# from scipy.stats import *

pd.set_option('display.max_columns', None)

<IPython.core.display.Javascript object>

In [13]:
from sklearn.preprocessing import StandardScaler


In [2]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [3]:
np.set_printoptions(suppress=True)

In [4]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [7]:
def process_df(df, batter_list, pitcher_list):
    
    df = df[(df['pitcher'].isin(pitcher_list))&(df['batter'].isin(batter_list))].copy()
    df['pitcher'] = df['pitcher'].astype('category')
    df['batter'] = df['batter'].astype('category')
    df['ump_id'] = df['ump_id'].astype('category')
    df['ump_id'] = df['ump_id'].astype('category')

    df['weather_cond'] = df['weather_cond'].str.replace(' ', '').copy()
    df['weather_cond'] = 'weather_cond_' + df['weather_cond']
    df['wind_direction'] = df['wind_direction'].str.replace(' ', '').copy()
    df['wind_direction'] = 'wind_dir_' + df['wind_direction']
    df['roof_cond'] = 'roof_cond_' + df['roof_cond']

    df = pd.concat([df, pd.get_dummies(df['weather_cond'])], axis=1)
    df = pd.concat([df, pd.get_dummies(df['wind_direction'])], axis=1)
    df = pd.concat([df, pd.get_dummies(df['roof_cond'])], axis=1)
    
    return df

In [5]:
T_batters_dict_weighted = joblib.load('data/T/T_batters_dict_1518_weighted')
T_pitchers_dict_weighted = joblib.load('data/T/T_pitchers_dict_1518_weighted')

batters_quali_list = list(T_batters_dict_weighted.keys())
pitchers_quali_list = list(T_pitchers_dict_weighted.keys())

In [51]:
data_df = pd.read_csv('data/PA_data_merged_15-19_012323.csv', index_col=[0])

In [53]:
data_df.shape

(924744, 36)

In [54]:
data_df = process_df(data_df, batter_list = batters_quali_list, pitcher_list = pitchers_quali_list)

In [56]:
data_df = main.test_T_dict(data_df, T_batters_dict_weighted, T_pitchers_dict_weighted, p_weight=0.34, _return_arr=False, is_pitch_data=False)


  0%|          | 0/137102 [00:00<?, ?it/s]

In [58]:
test_df = data_df[data_df['game_year']==2019].copy()

In [59]:
train_val_df = data_df[data_df['game_year'].isin([2015, 2016, 2017, 2018])].copy()

In [60]:
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=0, shuffle=True)

In [61]:
print(f"train size: {train_df.shape[0]}, {train_df.shape[0]/data_df.shape[0]:.0%}")
print(f"val size: {val_df.shape[0]}, {val_df.shape[0]/data_df.shape[0]:.0%}")
print(f"test size: {test_df.shape[0]}, {test_df.shape[0]/data_df.shape[0]:.0%}")

train size: 408501, 65%
val size: 136168, 22%
test size: 83661, 13%


In [62]:
scaler = StandardScaler()

In [63]:
scale_col = ['temp', 'wind_speed', '1B_pf', '2B_pf', '3B_pf', 'HR_pf', 'BB_pf', 'K_pf', 'sprint_speed']
scaled_arr = scaler.fit_transform(train_df[scale_col])

df_std = pd.DataFrame(scaled_arr, columns=[col + '_std' for col in scale_col])
train_df = pd.concat([train_df, df_std], axis=1)

In [64]:
val_arr = scaler.transform(val_df[scale_col])
val_df_std = pd.DataFrame(val_arr, columns=[col + '_std' for col in scale_col])
val_df = pd.concat([val_df, val_df_std], axis=1)

In [65]:
test_arr = scaler.transform(test_df[scale_col])
test_df_std = pd.DataFrame(test_arr, columns=[col + '_std' for col in scale_col])
test_df = pd.concat([test_df, test_df_std], axis=1)

In [66]:
# train_df.to_csv('R/Train_std_012523.csv')
# val_df.to_csv('R/Val_std_012523.csv')
# test_df.to_csv('R/Test_std_012523.csv')